In [ ]:
# install semantic-link-labs
%pip install semantic-link-labs

In [ ]:
Param_ConnectionId= ""
Param_ConnectionName= ""

In [ ]:
import sempy.fabric as fabric
import sempy_labs as labs
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import networkx as nx
import matplotlib.pyplot as plt



# ---------------------------
# Get List of all workspaces
# ---------------------------
df_workspaces = fabric.list_workspaces().query('Type != "AdminInsights"')

print(f"Found {len(df_workspaces)} workspaces")

# ---------------------------
# Get all items from these workspaces
# ---------------------------
df_items = pd.concat(
    [fabric.list_items(workspace=ws_id) for ws_id in df_workspaces.Id],
    ignore_index=True
)

print(f"Found {len(df_items)} items across all workspaces.")

#df_connections_items = df_items[df_items["Type"].isin(['Dataflow', 'DataPipeline', 'SemanticModel','Lakehouse','CopyJob'])] # Only these Item Types leverage Connections
df_connections_items = df_items[df_items["Type"].isin(['SemanticModel','Dataflow'])] 
df_connections_items = df_connections_items.merge(df_workspaces[["Id", "Name"]], left_on="Workspace Id", right_on="Id", suffixes=('', '_Workspace'))

# ---------------------------
#Iterate items and fetch connection details
# ---------------------------
connection_records = []

for _, item in df_connections_items.iterrows():
    try:
        connections_df = labs.list_item_connections(
            item_name=item["Display Name"],
            item_type=item.Type,
            workspace=item["Workspace Id"]
        )

        if not connections_df.empty:
            for _, conn in connections_df.iterrows():
                connection_records.append({
                    "WorkspaceId": item["Workspace Id"],
                    "WorkspaceName": item["Name"],
                    "ItemName": item["Display Name"],
                    "ItemType": item.Type,
                    "ConnectionId": conn.get("Connection Id", None),
                    "ConnectionName": conn.get("Connection Name", None),
                    "ConnectionType": conn.get("Connection Type", None)
                })

    except Exception as e:
        print(f"Failed to get connections for {item['Display Name']} of {item.Type} from Workspace {item['Name']}")

# ---------------------------
# Convert to DataFrame and display
# ---------------------------
df_connections = pd.DataFrame(connection_records)

# ---------------------------
# Filter based on Parameter Value
# ---------------------------
if Param_ConnectionId.strip(): 
    df_connections = df_connections[df_connections["ConnectionId"] == Param_ConnectionId]
else:
    if Param_ConnectionName.strip():
        df_connections = df_connections[df_connections["ConnectionName"] == Param_ConnectionName]
    else:
        df_connections = df_connections #Get all Connection Details

print(f"Total connections fetched: {len(df_connections['ConnectionId'].unique())}")

display(df_connections)

# ---------------------------
# Create DAG using networkx
# ---------------------------
G = nx.DiGraph()
node_colors = {}

for _, row in df_connections.iterrows():
    source_node = f"{row['ItemName']} ({row['ItemType']}) ({row['WorkspaceName']})"
    target_node = f"{row['ConnectionName']} ({row['ConnectionId']}) ({row['ConnectionType']})"
    
    G.add_edge(source_node, target_node)
    
    # Assign colors
    node_colors[source_node] = 'lightblue'   # sources in blue
    node_colors[target_node] = 'lightgreen'  # connections in green

# ---------------------------
#Plot the DAG
# ---------------------------

plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.6, seed=42)

# ---------------------------
# Draw nodes with custom colors
# ---------------------------

nx.draw(
    G, pos,
    with_labels=True,
    node_color=[node_colors[node] for node in G.nodes()],
    node_size=3000,
    font_size=8,
    font_weight='bold',
    arrows=True
)

plt.title("Connection Dependency DAG", fontsize=16)
plt.axis('off')
plt.show()
